In [3]:
import pandas as pd
import numpy as np
import dask.dataframe as dd

In [4]:
import psutil

for proc in psutil.process_iter():
    try:
        if proc.name() in ['dask-scheduler', 'dask-worker']:
            proc.terminate()
    except psutil.NoSuchProcess:
        pass

In [5]:
import os

os.environ['DASK_TEMPORARY_DIRECTORY'] = 'D:/dask_temp'
from dask.distributed import Client, LocalCluster

cluster = LocalCluster(
    local_directory='D:/dask_temp',
    n_workers=4,
    threads_per_worker=2,
    memory_limit='8GB'
)

client = Client(cluster)

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'D:\\'

In [6]:
ddf = dd.read_csv("C:/Users/aniru/OneDrive/Desktop/ges-hackathon/ges24/data/decrypted_file/train/train_data_feeds.csv", dtype={'u_newsCatDislike': 'object'})

In [7]:
dda = dd.read_csv("C:/Users/aniru/OneDrive/Desktop/ges-hackathon/ges24/data/decrypted_file/train/train_data_ads.csv", dtype={'ad_close_list_v001': 'object',
       'ad_close_list_v002': 'object',
       'ad_close_list_v003': 'object'})

In [8]:
ddf.head()

,u_userId,u_phonePrice,u_browserLifeCycle,u_browserMode,u_feedLifeCycle,u_refreshTimes,u_newsCatInterests,u_newsCatDislike,u_newsCatInterestsST,u_click_ca2_news,...,e_ch,e_m,e_po,e_pl,e_rn,e_section,e_et,label,cillabel,pro
0,135880,16,17,10,17,0,195^168^109^98^108,0,195^44^168^112^21,195^168^44^112^21,...,19,1217,1,561,2,0,202206081521,-1,-1,0
1,135880,16,17,10,17,0,195^168^109^98^108,0,195^44^168^112^21,195^168^44^112^21,...,19,1217,9,561,1,0,202206081521,-1,-1,0
2,135880,16,17,10,17,0,195^168^109^98^108,0,195^44^168^112^21,195^168^44^112^21,...,19,1217,18,561,1,0,202206081521,-1,-1,0
3,135880,16,17,10,17,0,195^168^109^98^108,0,195^44^168^112^21,195^168^44^112^21,...,19,1217,7,561,1,1,202206081521,-1,-1,0
4,135880,16,17,10,17,0,195^168^109^98^108,0,195^44^168^112^21,195^168^44^112^21,...,19,1217,7,561,2,0,202206081522,-1,-1,0


In [9]:
dda.head()

,log_id,label,user_id,age,gender,residence,city,city_rank,series_dev,series_group,...,ad_click_list_v001,ad_click_list_v002,ad_click_list_v003,ad_close_list_v001,ad_close_list_v002,ad_close_list_v003,pt_d,u_newsCatInterestsST,u_refreshTimes,u_feedLifeCycle
0,373250,0,100005,3,2,16,147,2,32,6,...,30157^30648^14278^31706,2066^1776^1036,114^219^312,24107,1218,173,202206030326,39^220^16,0,15
1,373253,1,100005,3,2,16,147,2,32,6,...,30157^30648^14278^31706,2066^1776^1036,114^219^312,24107,1218,173,202206030326,39^220^16,0,15
2,373252,1,100005,3,2,16,147,2,32,6,...,30157^30648^14278^31706,2066^1776^1036,114^219^312,24107,1218,173,202206030326,39^220^16,0,15
3,373251,0,100005,3,2,16,147,2,32,6,...,30157^30648^14278^31706,2066^1776^1036,114^219^312,24107,1218,173,202206030326,39^220^16,0,15
4,373255,0,100005,3,2,16,147,2,32,6,...,30157^30648^14278^31706,2066^1776^1036,114^219^312,24107,1218,173,202206030328,39^220^16,0,15


In [10]:
ddf.compute()['u_userId'].unique()

array([135880, 116738, 109694, ..., 216655, 127804, 137663], dtype=int64)

In [7]:
dda.compute()['user_id'].unique()

array([100005, 100006, 100009, ..., 286534, 286715, 286999], dtype=int64)

In [8]:
ddf.compute().shape

(3227732, 28)

In [9]:
dda.compute().shape

(7675517, 35)

In [11]:
ddf['user_id'] = ddf['u_userId']

In [12]:
ddf = ddf.drop('u_userId', axis=1)

In [13]:
merged_df = ddf.merge(dda, on='user_id', how='outer')

In [14]:
partition_df = merged_df.repartition(npartitions=1000)

persisted_df = partition_df.persist()

persisted_df.head()

MemoryError: Unable to allocate 262. MiB for an array with shape (34292832, 1) and data type float64

In [13]:
persisted_df.compute().shape

KeyboardInterrupt: 